In [2]:
import numpy as np

In [2]:
a = np.identity(3)

In [5]:
np.add(a,3)

array([[ 4.,  3.,  3.],
       [ 3.,  4.,  3.],
       [ 3.,  3.,  4.]])

In [11]:
b = np.vstack((np.array([1,2]).reshape(-1,1),1))

In [3]:
a = np.arange(0,10,1)

In [6]:
a[1:]

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

In [10]:
for t, i in enumerate(a[1:]):
    print(t)
    print(i)

0
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9


In [8]:
zip(a)